# 1. 单局 PK 可视化区域

In [1]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple, GomokuEnv
from gomoku.policy import ZeroPolicy

/home/smokingmouse/python/ai/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-24 17:15:06,414	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
ZeroPolicy(board_size=9)

ZeroPolicy(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res_blocks): ModuleList(
    (0-3): 4 x ResBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (policy_conv): Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1))
  (policy_bn): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (policy_fc): Linear(in_features=162, out_features=81, bias=True)
  (value_conv): Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1))
  (value_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (value_fc1): Linear(in_fe

In [6]:
from gomoku.player import IneffectiveZeroMCTSPlayer
import torch

board_size = 9

policy = ZeroPolicy(board_size)
policy.load_state_dict(torch.load('models/gomoku_zero_9_plus_pro_max/policy_step_87000.pth'))
player1 = ZeroMCTSPlayer(policy)
player2 = ZeroMCTSPlayer(policy)

game = GomokuEnv(board_size)

info = play_one_game(player1, player2, game=game,board_size=board_size, render=True, eager=False, itermax=400)

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  .  .  .  .  . 
 4 .  .  .  .  .  X  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  O  .  .  .  . 
 4 .  .  .  .  .  X  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  O  .  .  .  . 
 4 .  .  .  .  .  X  .  .  . 
 5 .  .  .  .  X  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  . 

In [ ]:
import cProfile
import torch
import pstats
import io

# --- 开始性能分析 ---

# 1. 创建一个 Profiler 对象
profiler = cProfile.Profile()

# 2. 启用 Profiler 并运行你的函数
profiler.enable()
policy = ZeroPolicy(board_size=15)
policy.load_state_dict(torch.load('models/gomoku_zero_15_continue/policy_step_6000.pth'))
player1 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)
player2 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)

game = GomokuEnv(15)

info = play_one_game(player1, player2, board_size=15, render=True)

profiler.disable()

# 3. 创建一个 IO 流来捕获分析结果
s = io.StringIO()

# 4. 创建 pstats.Stats 对象来格式化和排序结果
#    sort_stats() 的参数是排序依据，'cumulative' 是按累计耗时排序
stats = pstats.Stats(profiler, stream=s).sort_stats('cumulative')

# 5. 打印分析报告
stats.print_stats()

# 6. (可选) 只打印前 10 个最耗时的函数
# stats.print_stats(10)

print(s.getvalue())


# 2. ARENA 

In [34]:
from gomoku.player import arena_parallel
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

In [ ]:
policy1 = ZeroPolicy(board_size)
policy2 = ZeroPolicy(board_size)
policy1.load_state_dict(torch.load('models/gomoku_zero_9_plus_pro_max/policy_step_199000.pth'))
policy2.load_state_dict(torch.load('models/gomoku_zero_9_plus_pro_max/policy_step_199000.pth')) 

r = arena_parallel(
    policy1,
    policy2, 
    board_size=9,
    num_cpus=16,
    games=100,
    itermax=100,
)

Starting parallel arena with 100 games on 16 CPUs...


In [ ]:
from worker import gather_selfplay_games


gather_selfplay_games(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    num_workers=10,
    games_per_worker=10,
    itermax=100,
)

In [ ]:
from player import self_play


_ = self_play(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    itermax=100,
)

In [ ]:
_ = play_one_game(
    player1=ZeroMCTSPlayer(ZeroPolicy(board_size=9)),
    player2=ZeroMCTSPlayer(ZeroPolicy(board_size=9)),
    board_size=9,
    # render=True,
    game=GomokuEnvSimple(board_size=9),
    itermax=100
)

# 3. 评价棋局

In [11]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game, self_play
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy
import torch

In [ ]:
board_size = 9

policy = ZeroPolicy(board_size)
policy.load_state_dict(torch.load('models/gomoku_zero_9_plus_pro_max/policy_step_199500.pth'))

<All keys matched successfully>

In [22]:
infos = self_play(policy, 'cpu', board_size=board_size, itermax=400)


函数 'play_one_game' 执行耗时: 23.1266 秒
Game over! Winner: 1


In [23]:
def render(states, index):
    import numpy as np
    states_tensor = torch.from_numpy(np.array(infos['states'], dtype=np.float32))
    probs, winrate = policy(states_tensor)

    symbols = {0: '.', 1: 'X', 2: 'O'}
    board_str = "  " + " ".join([f"{i:2d}" for i in range(9)]) + "\n"
    
    for i in range(9):
        cells = []
        for j in range(9):
            if states[index][ 0, i, j] ==  1:  
                cell = 1
            elif states[index][ 1, i, j] == 1:
                cell = 2
            else:
                cell = 0
            cells.append(cell)
        board_str += f"{i:2d} " + " ".join([f"{symbols[cell]} " for cell in cells]) + "\n"
    print(board_str)
    print(winrate[index])
    values, indices = torch.topk(torch.softmax(probs, dim=-1)[index], 10)

    def action_2_index(action):
        return action // 9, action % 9

    for i  in range(len(indices)):
        a, b = action_2_index(indices[i].item())
        print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


In [32]:
render(infos['states'], -8)

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  X  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  O  .  .  .  . 
 3 .  .  .  .  .  X  .  .  . 
 4 .  .  .  .  O  O  X  .  . 
 5 .  .  .  .  O  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

tensor([-0.9361], grad_fn=<SelectBackward0>)
[ 1, 4]:0.0712
[ 7, 4]:0.0629
[ 3, 3]:0.0567
[ 5, 5]:0.0393
[ 3, 6]:0.0318
[ 5, 2]:0.0296
[ 2, 5]:0.0246
[ 6, 3]:0.0244
[ 3, 4]:0.0237
[ 3, 1]:0.0235


In [ ]:
render(info[1]['states'], -2)

In [ ]:
def action_2_index(action):
    return action // 9, action % 9

probs = torch.tensor(info[1]['probs'])
values, indices = probs[-2].topk(10)

for i  in range(len(indices)):
    a, b = action_2_index(indices[i].item())
    print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")
